In [1]:
import numpy as np

In [2]:
np.set_printoptions(precision=4, suppress=True)

============================================================
1. HIGH-DIMENSIONAL DATA GENERATION
============================================================

In [3]:
np.random.seed(42)

In [4]:
n = 60
d = 500
s = 8
sigma = 0.1

In [5]:
X = np.random.randn(n, d)

In [6]:
beta_true = np.zeros(d)
beta_true[:s] = np.random.randn(s)

In [7]:
y = X @ beta_true + sigma * np.random.randn(n)

In [8]:
print("High-dimensional data generated")
print(f"X shape: {X.shape}")
print("-" * 60)

High-dimensional data generated
X shape: (60, 500)
------------------------------------------------------------


============================================================
2. EIGEN-DECOMPOSITION OF XᵀX (CORE SVD STEP)
============================================================

In [9]:
XtX = X.T @ X

In [10]:
# Since XᵀX is symmetric positive semidefinite
eigvals, V = np.linalg.eigh(XtX)

In [11]:
# Sort eigenvalues descending
idx = np.argsort(eigvals)[::-1]
eigvals = eigvals[idx]
V = V[:, idx]

In [12]:
# Keep only non-zero eigenvalues
tol = 1e-10
mask = eigvals > tol

In [13]:
eigvals = eigvals[mask]
V = V[:, mask]

In [14]:
# Singular values
S = np.sqrt(eigvals)

In [15]:
print("Number of non-zero singular values:", len(S))
print("Expected rank (≤ n):", n)
print("-" * 60)

Number of non-zero singular values: 60
Expected rank (≤ n): 60
------------------------------------------------------------


============================================================
3. CONSTRUCT LEFT SINGULAR VECTORS U
============================================================

In [16]:
U = X @ V / S

============================================================
4. VERIFY SVD PROPERTIES
============================================================

In [17]:
print("UᵀU ≈ I:", np.allclose(U.T @ U, np.eye(len(S)), atol=1e-8))
print("VᵀV ≈ I:", np.allclose(V.T @ V, np.eye(len(S)), atol=1e-8))
print("-" * 60)

UᵀU ≈ I: True
VᵀV ≈ I: True
------------------------------------------------------------


============================================================
5. RECONSTRUCTION CHECK
============================================================

In [18]:
X_recon = U @ np.diag(S) @ V.T
recon_error = np.linalg.norm(X - X_recon)

In [19]:
print("Reconstruction error ||X − UΣVᵀ||₂:", recon_error)
print("-" * 60)

Reconstruction error ||X − UΣVᵀ||₂: 2.835140163384175e-13
------------------------------------------------------------


============================================================
6. RIDGE REGRESSION VIA *CUSTOM* SVD
============================================================

In [20]:
def ridge_from_svd(U, S, V, y, lam):
    """
    Ridge regression using explicit SVD:

        β̂ = V diag( s_i / (s_i^2 + λ) ) Uᵀ y
    """
    shrinkage = S / (S**2 + lam)
    return V @ (shrinkage * (U.T @ y))

In [21]:
lam = 1.0
beta_ridge_svd = ridge_from_svd(U, S, V, y, lam)

In [22]:
print("Ridge solution via SVD computed")
print("||β̂||₂:", np.linalg.norm(beta_ridge_svd))
print("-" * 60)

Ridge solution via SVD computed
||β̂||₂: 1.083229279676445
------------------------------------------------------------


============================================================
7. COMPARE WITH PRIMAL RIDGE SOLUTION
============================================================

In [23]:
beta_primal = np.linalg.solve(
    X.T @ X + lam * np.eye(d),
    X.T @ y
)

In [24]:
diff = np.linalg.norm(beta_primal - beta_ridge_svd)

In [25]:
print("||β_primal − β_svd||₂:", diff)
print("→ Solutions are numerically identical")
print("-" * 60)

||β_primal − β_svd||₂: 1.292253286401615e-13
→ Solutions are numerically identical
------------------------------------------------------------


============================================================
8. ESTIMATION ERROR
============================================================

In [26]:
err = np.linalg.norm(beta_ridge_svd - beta_true)

In [27]:
print("Estimation error ||β̂ − β*||₂:", err)
print("-" * 60)

Estimation error ||β̂ − β*||₂: 2.698164401161711
------------------------------------------------------------


============================================================
9. EFFECTIVE DIMENSION (SVD VIEW)
============================================================

In [28]:
def effective_dimension(S, lam):
    return np.sum(S**2 / (S**2 + lam))

In [29]:
for lam_test in [0.01, 0.1, 1.0, 10.0, 100.0]:
    d_eff = effective_dimension(S, lam_test)
    print(f"λ = {lam_test:<6} → effective dimension = {d_eff:.2f}")

λ = 0.01   → effective dimension = 60.00
λ = 0.1    → effective dimension = 59.99
λ = 1.0    → effective dimension = 59.86
λ = 10.0   → effective dimension = 58.66
λ = 100.0  → effective dimension = 49.05


In [30]:
print("-" * 60)
print("Custom SVD + ridge regression completed successfully.")

------------------------------------------------------------
Custom SVD + ridge regression completed successfully.
